In [43]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import os
import env

In [44]:
# Getting conncection to mySQL database, and acquiring data
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''
    This function gets conncection to mySQL database
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

# Loading raw data from Curriculum_Log database
def new_data():
    '''
    This function reads the curriculum data from the mySQL database into a df.
    '''
    # Create SQL query.
    sql_query = '''
    SELECT * FROM logs LEFT JOIN cohorts ON logs.user_id = cohorts.id
    '''
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('curriculum_logs'))
    
    return df

def get_data():
    '''
    This function reads in curriculum data from curriculum_log database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('curriculum.csv'):
        
        # If csv file exists, read in data from csv file.
        df = pd.read_csv('curriculum.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame.
        df = new_data()
        
        # Write DataFrame to a csv file.
        df.to_csv('curriculum.csv')
        
    return df

In [45]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = """SELECT * FROM logs LEFT JOIN cohorts ON logs.user_id = cohorts.id"""
df = pd.read_sql(query, url)
# df.to_csv(cirriculum.csv, index=False)
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2.0,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          73739 non-null   float64
 7   name        73739 non-null   object 
 8   slack       73739 non-null   object 
 9   start_date  73739 non-null   object 
 10  end_date    73739 non-null   object 
 11  created_at  73739 non-null   object 
 12  updated_at  73739 non-null   object 
 13  deleted_at  0 non-null       object 
 14  program_id  73739 non-null   float64
dtypes: float64(3), int64(1), object(11)
memory usage: 103.0+ MB


In [47]:
# Drop deleted_at column


#df = df.drop(columns= 'deleted_at', inplace = True)
#df.head()

## Detecting Anomalies in Discrete Variables

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

---

#### Okay so how do I best approach this problem?

1. Create and separate the datasets of Web Development and Data Science programs.
    - 1 == WD
    - 2 == WD
    - 3 == DS
    - 4 == WD
    
2. Remove the popular logistic pages in the curriculum. I.E. '/' = main page, 'toc' = Table of Contents, et.c
3. Use .value_counts() on path to see the most accessed pages.
4. Store the top 10 page names with page counts as a variable.

In [48]:
df.path.value_counts()

/                                                               50313
search/search_index.json                                        19519
javascript-i                                                    18983
toc                                                             18297
java-iii                                                        13733
                                                                ...  
javascript/loops                                                    1
content/control-structures-ii                                       1
javascript-ii/promisesdfghjkjhgfs                                   1
app                                                                 1
appendix/professional-development/post-interview-review-form        1
Name: path, Length: 2313, dtype: int64

In [49]:
df.path.value_counts(normalize=True).head(20)

/                                                                            0.055890
search/search_index.json                                                     0.021682
javascript-i                                                                 0.021087
toc                                                                          0.020325
java-iii                                                                     0.015255
html-css                                                                     0.015146
java-ii                                                                      0.014091
spring                                                                       0.013912
jquery                                                                       0.012802
mysql                                                                        0.012256
java-i                                                                       0.012069
javascript-ii                                         

In [7]:
# how many unique paths we have in the dataset?
df.path.nunique()

2313

In [8]:
df.nunique()

date           1182
time          73167
path           2313
user_id         981
cohort_id        47
ip             5531
id               53
name             53
slack            52
start_date       49
end_date         50
created_at       41
updated_at       41
deleted_at        0
program_id        4
dtype: int64

In [22]:
df.loc[df.program_id==1].path.value_counts().nlargest(10)

/                                           3513
javascript-i                                 605
classification/overview                      584
sql/mysql-overview                           554
java-iii                                     499
6-regression/1-overview                      489
spring                                       477
java-ii                                      461
search/search_index.json                     447
classification/scale_features_or_not.svg     441
Name: path, dtype: int64

In [23]:
df.loc[df.program_id==2].path.value_counts().nlargest(10)

index.html      1520
/               1353
spring           825
javascript-i     759
java-iii         687
java-ii          594
html-css         555
appendix         522
mysql            484
java-i           464
Name: path, dtype: int64

In [24]:
df.loc[df.program_id==3].path.value_counts().nlargest(10)

appendix                                                                     112
mkdocs/search_index.json                                                     110
javascript-i                                                                  76
javascript-i/loops                                                            51
jquery                                                                        45
javascript-i/arrays                                                           42
javascript-i/conditionals                                                     41
javascript-i/javascript-with-html                                             40
javascript-i/introduction/working-with-data-types-operators-and-variables     39
/                                                                             39
Name: path, dtype: int64

In [25]:
df.loc[df.program_id==4].path.value_counts().nlargest(10)

/                                              151
spring                                          68
javascript-i                                    46
jquery                                          42
java-i                                          38
java-ii                                         36
javascript-ii                                   35
mysql                                           28
appendix                                        22
spring/fundamentals/security/authentication     20
Name: path, dtype: int64

In [41]:
# Remove staff from the analysis.
df.loc[~(df.name == 'Staff')]

# Step 1.
# Web Devlopment Cohorts
wd_cohorts = df.loc[df.program_id.isin([1, 2, 4])]

# Data Science Cohorts
ds_cohorts = df.loc[df.program_id == 3]

AttributeError: 'NoneType' object has no attribute 'loc'